In [26]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, T5ForConditionalGeneration

In [27]:
data = pd.read_csv('data/ProblemSolutionPythonV3.csv')

In [28]:
data

,Unnamed: 0,Problem,Python Code
0,0,Write a NumPy program to repeat elements of an...,"import numpy as np\rx = np.repeat(3, 4)\rprint..."
1,1,Write a Python function to create and print a ...,def printValues():\n\tl = list()\n\tfor i in r...
2,2,Write a Python program to remove duplicates fr...,"import itertools\rnum = [[10, 20], [40], [30, ..."
3,3,Write a NumPy program to compute the x and y c...,import numpy as np\rimport matplotlib.pyplot a...
4,4,Write a Python program to alter a given SQLite...,import sqlite3\rfrom sqlite3 import Error\rdef...
...,...,...,...
3302,3302,Python Program to Check Whether a Number is Po...,"\nn=int(input(""Enter number: ""))\nif(n>0):\n ..."
3303,3303,\nThe Fibonacci Sequence is computed based on ...,\ndef f(n):\n if n == 0: return 0\n elif...
3304,3304,\n\n\nPlease raise a RuntimeError exception.\n:,\nraise RuntimeError('something wrong')\n\n\n\n
3305,3305,Program to print inverted right triangle alpha...,"\nprint(""Enter the row and column size:"");\nro..."


In [29]:
data = data.iloc[:, 1:]

In [30]:
data

,Problem,Python Code
0,Write a NumPy program to repeat elements of an...,"import numpy as np\rx = np.repeat(3, 4)\rprint..."
1,Write a Python function to create and print a ...,def printValues():\n\tl = list()\n\tfor i in r...
2,Write a Python program to remove duplicates fr...,"import itertools\rnum = [[10, 20], [40], [30, ..."
3,Write a NumPy program to compute the x and y c...,import numpy as np\rimport matplotlib.pyplot a...
4,Write a Python program to alter a given SQLite...,import sqlite3\rfrom sqlite3 import Error\rdef...
...,...,...
3302,Python Program to Check Whether a Number is Po...,"\nn=int(input(""Enter number: ""))\nif(n>0):\n ..."
3303,\nThe Fibonacci Sequence is computed based on ...,\ndef f(n):\n if n == 0: return 0\n elif...
3304,\n\n\nPlease raise a RuntimeError exception.\n:,\nraise RuntimeError('something wrong')\n\n\n\n
3305,Program to print inverted right triangle alpha...,"\nprint(""Enter the row and column size:"");\nro..."


In [31]:
len(data)

3307

In [32]:
data.iloc[2563, :]

Problem                  Program to convert Octal To Hexadecimal
Python Code    \ni=0\noctal=int(input("Enter Octal number:"))...
Name: 2563, dtype: object

In [33]:
train_data = pd.read_csv('data/ProblemSolutionPythonV3_TRAIN.csv').iloc[:, 1:]
test_data = pd.read_csv('data/ProblemSolutionPythonV3_TEST.csv').iloc[:, 1:]

In [34]:
test_data

,Problem,Python Code
0,Program to print the right triangle Alphabet p...,"\nprint(""Enter the row and column size:"");\nro..."
1,Python Program to Find the GCD of Two Numbers ...,"def gcd(a,b):\n if(b==0):\n return a..."
2,Python Program to Find Transitive Closure of a...,class Graph:\n def __init__(self):\n ...
3,Program to find addition of two matrices,"# Get size of matrix\nrow_size=int(input(""Ente..."
4,Remove all uppercase characters in the String,"\nstr=input(""Enter the String:"")\nstr2 = []\ni..."
...,...,...
500,Python Program to Check Whether a Number is Po...,"Â \nn=int(input(""Enter number: ""))\nif(n>0):\n..."
501,\nThe Fibonacci Sequence is computed based on ...,\ndef f(n):\n if n == 0: return 0\n elif...
502,\n\n\nPlease raise a RuntimeError exception.\n:,\nraise RuntimeError('something wrong')\n\n\n\n
503,Program to print inverted right triangle alpha...,"\nprint(""Enter the row and column size:"");\nro..."


In [18]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')

In [19]:
tokenizer.batch_encode_plus

{'input_ids': [1, 3067, 279, 6805, 9413, 5402, 358, 7666, 2186, 434, 392, 526, 18, 225, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
class NLtoPLDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length, max_target_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = self.data.iloc[index, 0]
        output_text = self.data.iloc[index, 1]

        input = self.tokenizer.batch_encode_plus([input_text], max_length=self.max_input_length,
                                                     padding='max_length',
                                                     truncation=True, return_tensors="pt")
        output = self.tokenizer.batch_encode_plus([output_text], max_length=self.max_target_length,
                                                      padding='max_length', truncation=True, return_tensors="pt")

        input_ids = input['input_ids'].squeeze()
        input_mask = input["attention_mask"].squeeze()
        output_ids = output["input_ids"].squeeze()
        output_mask = output["attention_mask"].squeeze()

        return {
            "input_ids": input_ids.to(dtype=torch.long),
            "input_mask": input_mask.to(dtype=torch.long),
            "output_ids": output_ids.to(dtype=torch.long),
            "output_mask": output_mask.to(dtype=torch.long)
        }

In [50]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    total_loss = 0
    for batch in loader:
        input_ids = batch['input_ids'].to(device, dtype=torch.long)
        input_mask = batch['input_mask'].to(device, dtype=torch.long)
        output_ids = batch['output_ids'].to(device, dtype=torch.long)
        y_ids = output_ids[:, :-1].contiguous()
        lm_labels = output_ids[:, 1:].clone().detach()
        lm_labels[output_ids[:, 1:] == tokenizer.pad_token_id] = -100

        outputs = model(
            input_ids=input_ids,
            attention_mask=input_mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )

        loss = outputs[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {total_loss / len(loader)}")

In [51]:
def val_test(tokenizer, model, device, loader, max_target_length):
    model.eval()
    predictions = []
    actual = []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device, dtype=torch.long)
            input_mask = batch['input_mask'].to(device, dtype=torch.long)
            y = batch['output_ids'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=input_mask,
                max_length=max_target_length,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]

            predictions.extend(preds)
            actual.extend(target)
    return predictions, actual

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [53]:
train_size = 0.8
train_dataset = train_data.sample(frac=train_size)
val_dataset = train_data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

In [54]:
train_ds = NLtoPLDataset(train_dataset, tokenizer, 512, 512)
val_ds = NLtoPLDataset(val_dataset, tokenizer, 512, 512)
test_ds = NLtoPLDataset(test_data, tokenizer, 512, 512)

In [55]:
train_loader = DataLoader(batch_size=8, dataset=train_ds, shuffle=True, num_workers=4)
val_loader = DataLoader(batch_size=1, dataset=val_ds, shuffle=False, num_workers=1)
test_loader = DataLoader(batch_size=1, dataset=test_ds, shuffle=False, num_workers=1)

In [56]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [58]:
torch.cuda.empty_cache()
for epoch in range(5):
    train(epoch, tokenizer, model, device, train_loader, optimizer)
    predictions, actual = val_test(tokenizer, model, device, val_loader, 512)
    print(predictions)
    print(actual)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.78 GiB total capacity; 6.20 GiB already allocated; 62.94 MiB free; 6.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [59]:
torch.cuda.empty_cache()